In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt

In [ ]:
path = '...' # enter path
img = plt.imread(path)
plt.imshow(img)

## Checking compressed images

In [ ]:
def plot_compressed_images(image, function, value1, value2, name):
    plt.figure(figsize=(15, 10))

    compressed_image_1, size1 = function(image, value1)
    plt.subplot(1, 3, 1)
    plt.imshow(compressed_image_1)
    plt.title(f'{name} = {value1}')
    plt.axis('off')

    compressed_image_2, size2 = function(image, value2)
    plt.subplot(1, 3, 2)
    plt.imshow(compressed_image_2)
    plt.title(f'{name} = {value2}')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(image)
    plt.title('Original image')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    return compressed_image_1, size1, compressed_image_2, size2

## **SVD**

In [ ]:
def eigenvalue(A, v):
    '''
    Eigenvalue
    '''
    Av = np.dot(A, v)
    return np.dot(Av, v)

In [ ]:
def power_iteration(matrix):
    '''
    Power Iteration method
    '''
    At_A = np.dot(matrix.T, matrix)
    m, n = matrix.shape

    eigen_vector = np.ones(n) / np.sqrt(n)
    evigen_value = eigenvalue(At_A, eigen_vector)



    while True:
        xi = np.dot(At_A, eigen_vector)
        xi = xi / np.sqrt(np.dot(xi, xi))

        ev_new = eigenvalue(At_A, xi)
        if np.abs(evigen_value - ev_new) < 0.001:
            break

        eigen_vector = xi
        evigen_value = ev_new

    return eigen_vector, np.sqrt(evigen_value)


In [ ]:
def svd_power_iterations(matrix: np.array, k: int):
    '''
    SVD via power iterations method
    '''
    n, m = matrix.shape
    U = np.zeros((n, k))
    S = np.zeros((k, k))
    Vt = np.zeros((k, m))
    A = np.copy(matrix.astype('float64'))

    for i in range(k):
        v_i, sigma_i = power_iteration(A)
        if not sigma_i:
            break
        Av_i = np.dot(A, v_i)

        u_i = Av_i / sigma_i

        U[:, i] = u_i
        Vt[i, :] = v_i

        S[i, i] = sigma_i

        A -= sigma_i * np.outer(u_i, v_i)

    return U, S, Vt

In [ ]:
def SVD_compression(image, k):
    """
    Compression
    """
    B = image[:, :, 0]
    G = image[:, :, 1]
    R = image[:, :, 2]

    U_B, S_B, Vt_B = svd_power_iterations(B, k)
    U_G, S_G, Vt_G = svd_power_iterations(G, k)
    U_R, S_R, Vt_R = svd_power_iterations(R, k)

    svdkB = np.dot(U_B, np.dot(S_B, Vt_B))
    svdkG = np.dot(U_G, np.dot(S_G, Vt_G))
    svdkR = np.dot(U_R, np.dot(S_R, Vt_R))

    compressed_image = np.stack((svdkB, svdkG, svdkR), axis = -1)

    compressed_image = compressed_image/255
    size = (compressed_image.shape[0] + compressed_image.shape[1] + 1) * 3 * k
    return compressed_image, size

In [ ]:
_, svd_size1, _, svd_size2 = plot_compressed_images(img, SVD_compression, 20, 60, 'Rank')

#### SVD compression ratio

In [ ]:
lower_bound_k = upper_bound_k = compression_limit = 0
m = img.shape[0] # original image height
n = img.shape[1] # original image width
original_size = 3 * m * n


unit_size = 3* (n + m + 1) # size of 1-rank matrix in expansion

lower_bound_k = round(0.6 * original_size/unit_size)
upper_bound_k = round(0.8 * original_size/unit_size)
compression_limit = round(original_size/unit_size)

print("Range of k for 60-80% compression:", lower_bound_k, "-", upper_bound_k)
print("It makes no sense for compression with SVD for k more than:", compression_limit)

Range of k for 60-80% compression: 329 - 439
It makes no sense for compression with SVD for k more than: 548


In [ ]:
def find_rank(image, cr):
  n, m, ch = image.shape
  original_size = n * m * ch
  unit_size = 3* (n + m + 1)
  rank = round(cr/100 * original_size / unit_size)
  return rank

## **FFT**

In [ ]:
def next_power_of_2(x):
    return 1 if x == 0 else 2**(x - 1).bit_length()

def FFT(x):
    n = len(x)
    if n <= 1:
        return x
    else:
        if n % 2 > 0:
            x = np.append(x, np.zeros(next_power_of_2(n) - n))
            n = len(x)
        even = FFT(x[0::2])
        odd = FFT(x[1::2])
        t = [np.exp(-2j * np.pi * k / n) * (odd[k] if k < len(odd) else 0) for k in range(n // 2)]
        return [even[k] + t[k] for k in range(n // 2)] + [even[k] - t[k] for k in range(n // 2)]

def IFFT(x):
    x_conjugate = np.conjugate(x)
    y = FFT(x_conjugate)
    return np.conjugate(y) / len(x)

def compress_channel(channel, threshold_factor):
    padded_channel = np.array([np.pad(row, (0, next_power_of_2(len(row)) - len(row)), mode='constant') for row in channel])
    row_fft = np.array([FFT(row) for row in padded_channel])
    padded_row_fft = np.array([np.pad(row, (0, next_power_of_2(len(row)) - len(row)), mode='constant') for row in row_fft.T]).T
    column_fft = np.array([FFT(col) for col in padded_row_fft.T]).T
    abs_column_fft = np.abs(column_fft)
    threshold = np.max(abs_column_fft) * threshold_factor
    mask = abs_column_fft > threshold
    column_fft *= mask
    column_ifft = np.array([IFFT(col) for col in column_fft.T]).T
    row_ifft = np.array([IFFT(row) for row in column_ifft])
    return np.real(row_ifft)[:channel.shape[0], :channel.shape[1]], np.sum(mask)

def FFT_compression(img, threshold_factor):
    channels = cv2.split(img)
    compressed_channels = []
    size = 0
    for ch in channels:
      res, size1 = compress_channel(ch, threshold_factor)
      compressed_channels.append(res)
      size += size1
    compressed_image = cv2.merge(compressed_channels)
    return compressed_image/255, size


In [ ]:
_, fft_size1, _, fft_size2 = plot_compressed_images(img, FFT_compression, 0.05, 0.001, 'Threshold')

#### FFT compression ratio

In [ ]:
size = img.shape[0] * img.shape[1] * 3
print(f"Initial image size:{size}")
print(f"Image size after FFT compression:{fft_size2}")

cr = (fft_size2/ size) * 100
print(f"Threshold: {0.001}")
print(f"FFT compression ratio: {cr}%")

## **Wavelet**

In [ ]:
def process_channel(data, keep, wavelet='haar', level=3):
    coeffs = pywt.wavedec2(data, wavelet=wavelet, level=level)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    Csort = np.sort(np.abs(coeff_arr.reshape(-1)))
    thresh = Csort[int(np.floor((1 - keep) * len(Csort)))]
    ind = np.abs(coeff_arr) > thresh
    non_zero_count = np.sum(ind)

    Cfilt = coeff_arr * ind
    coeffs_filt = pywt.array_to_coeffs(Cfilt, coeff_slices, output_format='wavedec2')
    reconstructed = pywt.waverec2(coeffs_filt, wavelet=wavelet)

    return reconstructed, non_zero_count

def wavelet_compression(image, keep):
    channels = [image[:, :, i] for i in range(3)]
    reconstructed_channels = []
    comp_size = 0
    for chan in channels:
        comp_ch, size = process_channel(chan, keep)
        reconstructed_channels.append(comp_ch)
        comp_size += size

    reconstructed_image = np.stack(reconstructed_channels, axis=-1).astype(np.uint8)

    return reconstructed_image, comp_size

In [ ]:
_, wave_size1, _, wave_size2 = plot_compressed_images(img, wavelet_compression, 0.00955, 0.02, 'Threshold')

### Wavelet compression ratio

In [ ]:
size = img.shape[0] * img.shape[1] * 3
print(f"Initial image size:{size}")
print(f"Image size after Wavelet transform:{wave_size2}")

cr = (wave_size2/ size) * 100
print(f"Threshold: {0.02}")
print(f"Wavelet compression ratio: {cr}%")

## Metrics

#### SSIM

In [ ]:
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

#### MSE

In [ ]:
def mse(original, compressed):
  return np.mean((original - compressed) ** 2)

#### PSNR

In [ ]:
def psnr(original, compressed):
    max_pixel = 255.0
    mean_square_error = mse(original, compressed)
    return 20 * np.log10(max_pixel / np.sqrt(mean_square_error))

#### Results

In [ ]:
def calculate_metrics(original, compressed):
    mse_index = mse(original, compressed)
    psnr_index = psnr(original, compressed)
    ssim_index = ssim(original, compressed)
    metrics = {
        'MSE': mse_index,
        'PSNR': psnr_index,
        'SSIM': ssim_index,
    }
    return metrics

## SVD vs FFT vs Wavelet

In [ ]:
cr = ...
fft_threshold = ...
k = find_rank(img, cr)
svd_threshold = ...

In [ ]:
compressed_image_fft, size_fft = FFT_compression(img, fft_threshold)
compressed_image_svd, size_svd = SVD_compression(img, k)
compressed_image_wavelet, size_wavelet = wavelet_compression(img, svd_threshold)

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(1, 3, 1)
plt.imshow(compressed_image_svd)
plt.title(f'SVD')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(compressed_image_fft)
plt.title("FFT")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(compressed_image_wavelet)
plt.title('Wavelet transform')
plt.axis('off')


In [ ]:
print(f'Compression ratio ≈ {cr}%')
print('--------------------------')
print('SVD')
metrics_svd = calculate_metrics(img/255, compressed_image_svd)
print(metrics_svd)

print('--------------------------')
print('FFT')
metrics_fft = calculate_metrics(img/255, compressed_image_fft)
print(metrics_fft)

print('--------------------------')
print('Wavelet')
metrics_wavelet = calculate_metrics(img/255, compressed_image_wavelet/255)
print(metrics_wavelet)